In [1]:
#%% Import libraries
import numpy as np
from import_dataset import import_all_files
from group_data import get_data
from group_data import split_data
from group_data import crop_clips
from group_data import filter_clips
from librosa.feature import mfcc
from tqdm import tqdm
from sklearn import decomposition
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D #for 3D plotting
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import preprocessing


from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical
from keras.callbacks import History 
from keras.callbacks import EarlyStopping

In [2]:
!unzip /content/drive/MyDrive/archive_lungs.zip

Archive:  /content/drive/MyDrive/archive_lungs.zip
replace Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/101_1b1_Al_sc_Meditron.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/101_1b1_Al_sc_Meditron.txt  
  inflating: Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/101_1b1_Al_sc_Meditron.wav  
  inflating: Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/101_1b1_Pr_sc_Meditron.txt  
  inflating: Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/101_1b1_Pr_sc_Meditron.wav  
  inflating: Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/102_1b1_Ar_sc_Meditron.txt  
  inflating: Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/102_1b1_Ar_sc_Meditron.wav  
  inflating: Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/103_2b2_Ar_mc_Li

In [3]:
!mkdir input
!mv "/content/Respiratory_Sound_Database/Respiratory_Sound_Database" "/content/input"

mkdir: cannot create directory ‘input’: File exists
mv: cannot move '/content/Respiratory_Sound_Database/Respiratory_Sound_Database' to '/content/input/Respiratory_Sound_Database': Directory not empty


In [4]:
#%% Get clips (all at same sample rate for ease of use)
sr = 11100
#directory = "/Users/feres/SUMMER2020/ML_7641/Project/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files"
print("started")
#directory = "D:\\Google Drive\\Programs\\Jupyter\\Machine Learning\\project\\data\\audio_and_txt_files"
directory = "/content/input/Respiratory_Sound_Database/audio_and_txt_files/"
clips = import_all_files(directory,sr)

# %% Crop/filter clips
clips = crop_clips(clips,5,sr)
# clips = filter_clips(clips,5,6,sr)

Files to Clips:   0%|          | 0/920 [00:00<?, ?it/s]

started


Cropping clips: 100%|██████████| 6898/6898 [00:01<00:00, 5557.46it/s]


In [5]:
#%% Do mfcc on cropped audio
for clip in tqdm(clips,"Doing MFCC"):
    clip.mfcc = mfcc(y=clip.cropped_sound, sr=sr)
    clip.flattened_mfcc = clip.mfcc.flatten()

Doing MFCC: 100%|██████████| 6898/6898 [01:09<00:00, 98.81it/s]


In [6]:
#%% Separate data by class
data = get_data(clips, grouping="default", dtype="clip")

Grouping data by default: 100%|██████████| 6898/6898 [00:00<00:00, 620146.38it/s]


In [7]:
#%% Split data into training, testing, and validation sets/labels
for d in data:
    random.shuffle(d)
train_split, test_split, valid_split = split_data(data,train=0.7,test=0.2,valid=0.1)

train_data = []
test_data = []
valid_data = []
train_labels = []
test_labels = []
valid_labels = []

i = 0
for clips in tqdm(train_split, "Training split"):
    for clip in clips:
        train_data.append(clip.flattened_mfcc)
        train_labels.append(i)
    i += 1

i = 0      
for clips in tqdm(test_split,"Testing split"):
    for clip in clips:
        test_data.append(clip.flattened_mfcc)
        test_labels.append(i)
    i += 1
i = 0
for clips in tqdm(valid_split,"Validation split"):
    for clip in clips:
        valid_data.append(clip.flattened_mfcc)
        valid_labels.append(i)
    i += 1

Validation split: 100%|██████████| 4/4 [00:00<00:00, 2309.00it/s]


In [8]:
#%% Scaling the input to standardize features by removing the mean and scaling to unit variance
scaler = preprocessing.StandardScaler().fit(train_data)
train_data = scaler.transform(train_data)
valid_data=scaler.transform(valid_data)
test_data=scaler.transform(test_data)

In [9]:
#%% Applying PCA to reduce dimension while still keeping 99% of the original variance
#pca = decomposition.PCA(n_components=0.99, svd_solver = 'full') 
#pca.fit(train_data)
#train_data = pca.transform(train_data)   
#valid_data = pca.transform(valid_data) 
#test_data = pca.transform(test_data)
    
#%% Plots for visualization 
# # Plotting number of component vs explained variance
# plt.figure()
# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance');

# # Scatter plot of the classes with most informative 3 principle components
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(train_data[:, 0], train_data[:, 1], train_data[:, 2], c=train_labels[:,0])

#%% One-hot encoding of the labels
train_labels = to_categorical(train_labels)
valid_labels = to_categorical(valid_labels)
test_labels = to_categorical(test_labels)

train_data = train_data.reshape(train_data.shape[0],train_data.shape[1],1)
valid_data = valid_data.reshape(valid_data.shape[0],valid_data.shape[1],1)
test_data =test_data.reshape(test_data.shape[0],test_data.shape[1],1)


In [13]:
# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy, validX, validy):
	#history = History()
	es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1)
	verbose, epochs, batch_size = 10, 50, 32
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2] , trainy.shape[1]
	model = Sequential()
	model.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
	model.add(Conv1D(filters=64, kernel_size=5, activation='relu'))
	model.add(Dropout(0.5))
	model.add(MaxPooling1D(pool_size=2))
	model.add(Flatten())
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	#hist=model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, callbacks=[history], validation_split = 0.1)
	hist=model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, callbacks=[es], validation_data=(validX, validy))
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=10)
	return accuracy

In [14]:
# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = np.mean(scores), np.std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [18]:

def run_experiment( trainX, trainy, testX, testy,validX, validy, repeats=10):

	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy, validX, validy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

In [19]:
# run the experiment
run_experiment( train_data ,train_labels, test_data,test_labels ,valid_data, valid_labels, repeats=10)

Epoch 1/50
151/151 [==============================] - 36s 21ms/step - loss: 1.3111 - accuracy: 0.5157 - val_loss: 1.0429 - val_accuracy: 0.5671
Epoch 2/50
151/151 [==============================] - 3s 17ms/step - loss: 0.9646 - accuracy: 0.5978 - val_loss: 1.0508 - val_accuracy: 0.5815
Epoch 3/50
151/151 [==============================] - 3s 17ms/step - loss: 0.8744 - accuracy: 0.6482 - val_loss: 0.9721 - val_accuracy: 0.6133
Epoch 4/50
151/151 [==============================] - 3s 17ms/step - loss: 0.7073 - accuracy: 0.7242 - val_loss: 0.9448 - val_accuracy: 0.6205
Epoch 5/50
151/151 [==============================] - 3s 17ms/step - loss: 0.5701 - accuracy: 0.7889 - val_loss: 0.9688 - val_accuracy: 0.6335
Epoch 6/50
151/151 [==============================] - 3s 17ms/step - loss: 0.4728 - accuracy: 0.8147 - val_loss: 1.0037 - val_accuracy: 0.6407
Epoch 7/50
151/151 [==============================] - 3s 17ms/step - loss: 0.3379 - accuracy: 0.8736 - val_loss: 1.1263 - val_accuracy: 0.613